# 1. Idealista Data Cleaning

In [0]:
Idealista_MILAN = spark.table("silver.edgar_sarto_revenue.idealista_milan_joined")
Idealista_MILAN_1 = spark.table("silver.edgar_sarto_revenue.idealista_milan_joined_1")
Idealista_MILAN_2 = spark.table("silver.edgar_sarto_revenue.idealista_milan_joined_2")

IRM = Idealista_MILAN.toPandas()
IRM_1 = Idealista_MILAN_1.toPandas()
IRM_2 = Idealista_MILAN_2.toPandas()


In [0]:
import pandas as pd

IRM['City'] = 'Milan'
IRM_1['City'] = 'Milan'
IRM_2['City'] = 'Milan'

IR = pd.concat([IRM, IRM_1, IRM_2], ignore_index=True)



In [0]:
IR.head(1)

,Text,Text1,Text10,Text11,Text12,Text13,Text14,Text15,Text16,Text17,Text18,Text19,Text2,Text20,Text21,Text22,Text23,Text24,Text25,Text26,Text27,Text28,Text3,Text4,Text5,Text6,Text7,Text8,Text9,City
0,\nThe Best Rent - Appartamenti in Affitto Brev...,\n205\n,Con ascensore,3º piano,,"Classe energetica (D.L. 192 del 2005): (222,4...",Riscaldamento centralizzato: Gas,Ammobiliato e con cucina attrezzata,Costruito nel 1950,Orientamento sud,Buono stato,1 bagno,\nMilano\n,2 locali,70 m² commerciali,\nAria condizionata\n,,,\n70 m2\n,2.100,\n2 locali\n,\n3º piano con ascensore\n,\nZona Garibaldi - Porta Venezia\n,\nQuartiere Centrale\n,"\nVia Copernico, 49\n","30,00 €/m²",2.100 €/mese,,\nCauzione di 1 mese\n,Milan


In [0]:
IR = IR.iloc[:, [0, 1, 12, 22, 23, 24, 25, 26, 27, 28, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 29]]

In [0]:
IR.head(1)

,Text,Text1,Text2,Text3,Text4,Text5,Text6,Text7,Text8,Text9,Text10,Text11,Text12,Text13,Text14,Text15,Text16,Text17,Text18,Text19,Text20,Text21,Text22,Text23,Text24,Text25,Text26,Text27,Text28,City
0,\nThe Best Rent - Appartamenti in Affitto Brev...,\n205\n,\nMilano\n,\nZona Garibaldi - Porta Venezia\n,\nQuartiere Centrale\n,"\nVia Copernico, 49\n","30,00 €/m²",2.100 €/mese,,\nCauzione di 1 mese\n,Con ascensore,3º piano,,"Classe energetica (D.L. 192 del 2005): (222,4...",Riscaldamento centralizzato: Gas,Ammobiliato e con cucina attrezzata,Costruito nel 1950,Orientamento sud,Buono stato,1 bagno,2 locali,70 m² commerciali,\nAria condizionata\n,,,\n70 m2\n,2.100,\n2 locali\n,\n3º piano con ascensore\n,Milan


In [0]:
## DROP ROWS WITHOUT A PRICE
IR = IR.dropna(subset=['Text6'])

In [0]:
## CONVERT ALL STRINGS TEXT 1 TO UPPER AND CHANG THE COLUMN NAME TO AGENCY

IR['Text'] = IR['Text'].str.upper()
IR.rename(columns={'Text': 'Agency'}, inplace=True)
IR['Agency'].replace('\n', '', regex=True, inplace=True)
IR['Agency'].value_counts()

SPACEST.COM BY ROOMLESS                          482
SANTAMARTA IMMOBILI                               48
UNIPLACES CUSTOMER SERVICE                        48
ICON PROPERTIES                                   45
NOVECENTOCASE                                     39
                                                ... 
AFFILIATO TECNOCASA: A3G SAS                       1
AFFILIATO TECNOCASA: IMMOBILIARE MI.CA S.A.S.      1
FRANCIMMOBILIARE SAS DI ANNECCHINI & C.            1
TEMPOCASA MILANO - CERTOSA .                       1
VENDOCASA MILANO TIBALDI                           1
Name: Agency, Length: 940, dtype: int64

## 1.1. City / Zone / Neighborhood / Street

In [0]:
## CLEAN CITY COLUMN

#IR['Text2.1'] = IR['Text2']
#
#def shift_values(row):
#    if row['Text2'] is None:
#        row['Text2'] = 'Roma'
#    elif 'Zona' in row['Text2']:
#        row['Text2'] = None
#    elif 'Roma' not in row['Text2']:
#        row['Text2'] = 'Roma'
#    elif 'Roma, Roma' in row['Text2']:
#        row['Text2'] = 'Roma'
#    return row
#
#IR = IR.apply(shift_values, axis=1)
#
#IR.rename(columns={'Text2': 'City', 'Text2.1': 'Text2'}, inplace=True)

IR['City'] = IR['City'].str.upper()

IR['City'].value_counts()

MILAN    3438
Name: City, dtype: int64

In [0]:
IR['Text3']

0       \nZona Garibaldi - Porta Venezia\n
1       \nZona Garibaldi - Porta Venezia\n
2       \nZona Garibaldi - Porta Venezia\n
3               \nZona Navigli - Bocconi\n
4                  \nZona Porta Vittoria\n
                       ...                
3433            \nZona Navigli - Bocconi\n
3434               \nZona Porta Vittoria\n
3435               \nZona Porta Vittoria\n
3436            \nZona Navigli - Bocconi\n
3437            \nZona Navigli - Bocconi\n
Name: Text3, Length: 3438, dtype: object

In [0]:
## CLEAN ZONE COLUMN

IR['Text3.1'] = IR['Text3']

def shift_values(row):
    if pd.notna(row['Text3']) and 'Roma' in row['Text3']:
        row['Text3'] = None
    elif pd.notna(row['Text2']) and 'Zona' in row['Text2']:
        row['Text2'], row['Text3'] = None, row['Text2']
    elif pd.notna(row['Text3']) and 'Zona' not in row['Text3']:
        row['Text3'] = None
    return row

IR = IR.apply(shift_values, axis=1)

IR.rename(columns={'Text3': 'District', 'Text3.1': 'Text3'}, inplace=True)

IR.drop(columns=['Text2'], inplace=True)

IR['District'] = IR['District'].str.upper()

IR['District'].replace('\n', '', regex=True, inplace=True)

IR['District'].value_counts()

ZONA FIERA - DE ANGELI               894
ZONA GARIBALDI - PORTA VENEZIA       828
ZONA NAVIGLI - BOCCONI               677
ZONA PORTA VITTORIA                  459
ZONA CENTRO STORICO                  358
ZONA COMASINA - BICOCCA               31
ZONA GRECO - TURRO                    25
ZONA LORENTEGGIO - BANDE NERE         25
ZONA ABBIATEGRASSO - CHIESA ROSSA     14
ZONA CITTÀ STUDI - LAMBRATE           13
ZONA SAN SIRO - TRENNO                10
ZONA CERTOSA                           4
ZONA VIGENTINO - RIPAMONTI             2
ZONA FAMAGOSTA - NAVIGLIO GRANDE       1
Name: District, dtype: int64

In [0]:
## CLEAN NEIGHBORHOOD COLUMN

IR['Text4.1'] = IR['Text4']

def shift_values(row):
    if pd.notna(row['Text4']) and 'Zona' in row['Text4']:
        row['Text4'] = None
    elif pd.notna(row['Text3']) and 'Quartiere' in row['Text3']:
        row['Text3'], row['Text4'] = None, row['Text3']
    elif pd.notna(row['Text4']) and 'Quartiere' not in row['Text4']:
        row['Text4'] = None
    return row

IR = IR.apply(shift_values, axis=1)

IR.rename(columns={'Text4': 'Neighborhood', 'Text4.1': 'Text4'}, inplace=True)

IR.drop(columns=['Text3'], inplace=True)

IR['Neighborhood'] = IR['Neighborhood'].str.upper()

IR['Neighborhood'].replace('\n', '', regex=True, inplace=True)

IR['Neighborhood'].value_counts()

QUARTIERE CINQUE GIORNATE - CADORE                     225
QUARTIERE SAN VITTORE - WASHINGTON                     191
QUARTIERE SEMPIONE                                     165
QUARTIERE ISOLA                                        162
QUARTIERE NAVIGLI - PORTA GENOVA                       162
QUARTIERE ARIOSTO - MAGENTA                            143
QUARTIERE CARROBBIO - CINQUE VIE                       143
QUARTIERE SOLARI - SAVONA                              136
QUARTIERE GOTTARDO                                     128
QUARTIERE FIERA                                        125
QUARTIERE PORTA ROMANA                                 122
QUARTIERE CENTRALE                                     111
QUARTIERE INDIPENDENZA - REGINA GIOVANNA               109
QUARTIERE CORSO GENOVA                                 109
QUARTIERE TRIBUNALE - GUASTALLA                        106
QUARTIERE CENISIO - SARPI                               92
QUARTIERE PORTELLO - TRE TORRI                          

In [0]:
IR.rename(columns={'Text5': 'Street'}, inplace=True)
IR['Street'] = IR['Street'].str.upper()

## 1.2. Prices

In [0]:
## € / SQM2

def shift_values(row):
    if row['Text6'] is not None and 'piano con ascensore' in row['Text6']:
        row['Text6'], row['Text28'] = None, row['Text6']
    return row

IR = IR.apply(shift_values, axis=1)

IR.rename(columns={'Text6': '€_Sqm'}, inplace=True)

euro_sm2 = {' €/m²':'' ,' �/m_': '', ',':'.'}

IR['€_Sqm'].replace(euro_sm2, regex=True, inplace=True)

IR['€_Sqm'].replace('', float('nan'), inplace=True)

IR['€_Sqm'] = IR['€_Sqm'].astype(float)


In [0]:
## MONTHLY PRICE

price = {' €/mes': '', ' €/mese': '', 'e': ''}

IR.rename(columns={'Text7': 'Monthly_Price'}, inplace=True)

IR['Monthly_Price'].replace(price, regex=True, inplace=True)

IR['Monthly_Price'].replace('', float('nan'), inplace=True)

IR['Monthly_Price'] = IR['Monthly_Price'].astype(float)

def multiply_if_less_than_100(x):
    return x * 1000 if x < 100 else x

IR['Monthly_Price'] = IR['Monthly_Price'].apply(multiply_if_less_than_100)

In [0]:
IR

,Agency,Text1,District,Neighborhood,Street,€_Sqm,Monthly_Price,Text8,Text9,Text10,Text11,Text12,Text13,Text14,Text15,Text16,Text17,Text18,Text19,Text20,Text21,Text22,Text23,Text24,Text25,Text26,Text27,Text28,City,Text4
0,THE BEST RENT - APPARTAMENTI IN AFFITTO BREVE ...,\n205\n,ZONA GARIBALDI - PORTA VENEZIA,QUARTIERE CENTRALE,"\nVIA COPERNICO, 49\n",30.00,2100.0,,\nCauzione di 1 mese\n,Con ascensore,3º piano,,"Classe energetica (D.L. 192 del 2005): (222,4...",Riscaldamento centralizzato: Gas,Ammobiliato e con cucina attrezzata,Costruito nel 1950,Orientamento sud,Buono stato,1 bagno,2 locali,70 m² commerciali,\nAria condizionata\n,,,\n70 m2\n,2.100,\n2 locali\n,\n3º piano con ascensore\n,MILAN,\nQuartiere Centrale\n
1,QUIMMO PRESTIGE AGENCY,\n01-Fara135_2\n,ZONA GARIBALDI - PORTA VENEZIA,QUARTIERE CENTRALE,"\nVIA GUSTAVO FARA, 13\n",25.93,3500.0,,\n358 €/mese di spese condominiali\n,Con ascensore,3º piano,Riscaldamento autonomo,Cucina arredata e casa non arredata,Costruito nel 1970,Cantina,Buono stato,Garage incluso nel prezzo,Terrazzo,2 bagni,3 locali,135 m² commerciali,,,Occasione,\n135 m2\n,3.500,\n3 locali\n,\n3º piano con ascensore\n,MILAN,\nQuartiere Centrale\n
2,THE BEST RENT - APPARTAMENTI IN AFFITTO BREVE ...,\n163\n,ZONA GARIBALDI - PORTA VENEZIA,QUARTIERE PORTA VENEZIA - REPUBBLICA,"\nVIA LAZZARO PALAZZI, 4\n",50.00,2500.0,,\nCauzione di 1 mese\n,Con ascensore,2º piano,,,,Classe energetica (D.L. 192 del 2005): (IPE n...,Ammobiliato e con cucina attrezzata,Armadi a muro,Buono stato,1 bagno,2 locali,50 m² commerciali,\nAria condizionata\n,,,\n50 m2\n,2.500,\n2 locali\n,\n2º piano con ascensore\n,MILAN,\nQuartiere Porta Venezia - Repubblica\n
3,MILANO HOUSES,\nSOLARI 34\n,ZONA NAVIGLI - BOCCONI,QUARTIERE SOLARI - SAVONA,\nVIA ANDREA SOLARI\n,20.58,1070.0,\nCauzione di 2 mesi\n,\n180 €/mese di spese condominiali\n,Con ascensore,4º piano,,"Classe energetica (Legge 90 del 2013, legislaz...",Riscaldamento autonomo: Gas naturale,Ammobiliato e con cucina attrezzata,Costruito nel 1950,"Orientamento sud, est",Buono stato,1 bagno,2 locali,52 m² commerciali,,In Zona Solari -Tortona a pochi passi dalla MM...,Affitto stagionale,\n52 m2\n,1.070,\n2 locali\n,\n4º piano con ascensore\n,MILAN,\nQuartiere Solari - Savona\n
4,GIORGIO VIGANÒ SRL,\ngvr1680 Fontana\n,ZONA PORTA VITTORIA,QUARTIERE TRIBUNALE - GUASTALLA,\nVIA FONTANA S.N.C\n,17.90,2417.0,,\n375 €/mese di spese condominiali\n,Con ascensore,3º piano,,,"Classe energetica (Legge 90 del 2013, legislaz...",Riscaldamento centralizzato,Cucina non arredata e casa non arredata,Cantina,Buono stato,2 bagni,3 locali,135 m² commerciali,\nAria condizionata\n,,,\n135 m2\n,2.417,\n3 locali\n,\n3º piano con ascensore\n,MILAN,\nQuartiere Tribunale - Guastalla\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,TECNOSVILUPPO TIBALDI S.A.S. DI CRISTIAN FRAGA...,\n31724997\n,ZONA NAVIGLI - BOCCONI,QUARTIERE GOTTARDO,"\nVIA ANTONIO TANTARDINI, 22\n",13.27,1300.0,,\n200 €/mese di spese condominiali\n,Senza ascensore,Seminterrato,,,,Classe energetica: Non indicato,Riscaldamento centralizzato: Gas,Costruito nel 1930,Buono stato,1 bagno,2 locali,98 m² commerciali,\nAria condizionata\n,,,\n98 m2\n,1.300,\n2 locali\n,\nSeminterrato senza ascensore\n,MILAN,\nQuartiere Gottardo\n
3434,AGENZIA TROVA LA TUA CASA,\nMF10 R 07\n,ZONA PORTA VITTORIA,QUARTIERE CINQUE GIORNATE - CADORE,"\nVIA ANDREA MAFFEI, 10\n",20.00,1100.0,\nCauzione di 3 mesi\n,\n200 €/mese di spese condominiali\n,Con ascensore,1º piano,,Classe energetica (D.L. 192 del 2005): (IPE n...,Riscaldamento centralizzato: Gas,Ammobiliato e con cucina attrezzata,Costruito nel 1960,Buono stato,Garage e posto auto per 270 €/mese in più,1 bagno,1 locale,55 m² commerciali,\nAree verdi\n,,,\n55 m2\n,1.100,\n1 locale\n,\n1º piano con ascensore\n,MILAN,\nQuartiere Cinque Giornate - Cadore\n
3435,AGENZIA TROVA LA TUA CASA,\n30828343\n,ZONA PORTA VITTORIA,QUARTIERE CINQUE GIORNATE -

In [0]:
## ADDITIONAL COSTS

def shift_values(row):
    if row['Text9'] is not None and 'Cauzione' in row['Text9']:
        row['Text8'], row['Text9'] = row['Text9'], None
    return row

IR = IR.apply(shift_values, axis=1)

IR.rename(columns={'Text9': 'Additional_Costs', 'Text8': 'Security_Deposit'}, inplace=True)

additional_costs = {' €/mese di spese condominiali': ''}

IR['Additional_Costs'].replace(additional_costs, regex=True, inplace=True)

IR['Additional_Costs'].replace('', float('nan'), inplace=True)

IR['Additional_Costs'] = IR['Additional_Costs'].astype(float)


In [0]:
## SECURITY DEPOSIT PRICE

sec_dep = {'Cauzione di ': '', ' mesi':'', ' mese':'', ' o più':'', '\n':''}

IR['Security_Deposit'].replace(sec_dep, regex=True, inplace=True)

IR['Security_Deposit'] = IR['Security_Deposit'].replace('', float('nan'), inplace=True)

IR['Security_Deposit'] = IR['Monthly_Price'] * IR['Security_Deposit']


## 1.3. Bedrooms / Bathrooms / Size / Floor / Elevator

In [0]:
## BEDROOMS

IR['Text20.1'] = IR['Text20']

def shift_values(row):
    if pd.notna(row['Text20']) and ('locali' not in row['Text20'] and 'locale' not in row['Text20']):
        row['Text20'] = None
    if pd.notna(row['Text19']) and ('locali' in row['Text19'] or 'locale' in row['Text19']):
        row['Text19'], row['Text20'] = None, row['Text19']
    if pd.notna(row['Text18']) and ('locali' in row['Text18'] or 'locale' in row['Text18']):
        row['Text18'], row['Text20'] = None, row['Text18']
    return row

IR = IR.apply(shift_values, axis=1)

IR.rename(columns={'Text20': 'Bedrooms', 'Text20.1': 'Text20'}, inplace=True)

Bedrooms_clean = {' locale': '', ' locali': '', 'Senza':'0'}

IR['Bedrooms'].replace(Bedrooms_clean, regex=True, inplace=True)

IR['Bedrooms'].value_counts()

2     1851
3      822
1      393
4      205
5      107
6       32
7        9
0        8
8        5
10       3
26       1
9        1
Name: Bedrooms, dtype: int64

In [0]:
## BATHROOMS

IR['Text19.1'] = IR['Text19']

def shift_values(row):
    if pd.notna(row['Text19']) and ('bagno' not in row['Text19'] and 'bagni' not in row['Text19']):
        row['Text19'] = None
    if pd.notna(row['Text18']) and ('locali' in row['Text18'] or 'locale' in row['Text18']):
        row['Text18'], row['Text19'] = None, row['Text18']
    if pd.notna(row['Text17']) and ('locali' in row['Text17'] or 'locale' in row['Text17']):
        row['Text17'], row['Text19'] = None, row['Text17']
    return row

IR = IR.apply(shift_values, axis=1)

IR.rename(columns={'Text19': 'Bathrooms', 'Text19.1': 'Text19'}, inplace=True)

Bathrooms_clean = {' bagno': '', ' bagni': '', 'Senza':'0'}

IR['Bathrooms'].replace(Bathrooms_clean, regex=True, inplace=True)

IR['Bathrooms'].value_counts()

1     2586
2      681
3      142
4       18
5        5
7        2
19       1
Name: Bathrooms, dtype: int64

In [0]:
## SIZE

IR.rename(columns={'Text25': 'Constructed_Area'}, inplace=True)

Size_clean = {' m2': '', '\n':''}

IR['Constructed_Area'].replace(Size_clean, regex=True, inplace=True)

IR['Constructed_Area'].replace('', float('nan'), inplace=True)

In [0]:
IR['Constructed_Area'].astype(float)

0        70.0
1       135.0
2        50.0
3        52.0
4       135.0
        ...  
3433     98.0
3434     55.0
3435    116.0
3436     54.0
3437     58.0
Name: Constructed_Area, Length: 3438, dtype: float64

In [0]:
## ELEVATOR

def shift_values(row):
    if row['Text11'] is not None and ('ascensore' in row['Text11']):
        row['Text10'], row['Text11'] = row['Text11'], None
    return row

IR = IR.apply(shift_values, axis=1)

dummy_elevator = {'Con ascensore': '1', 'Senza ascensore': '0'}

IR['Text10'].replace(dummy_elevator, regex=True, inplace=True)

IR.rename(columns={'Text10': 'Elevator'}, inplace=True)

IR['Elevator'].value_counts()

1    2694
0     696
       48
Name: Elevator, dtype: int64

In [0]:
## FLOOR

def shift_values(row):
    if row['Text11'] is not None and ('ascensore' in row['Text11']):
        row['Text11'] = None
    return row

IR = IR.apply(shift_values, axis=1)

floor_clean = {'º piano': '', 'Piano terra': '0', 'Piano rialzato': '0', 'Seminterrato': '-1', 'Piano interrato': '-1', 'Piano ':''}

IR['Text11'].replace(floor_clean, regex=True, inplace=True)

IR.rename(columns={'Text11': 'Floor'}, inplace=True)

IR['Floor'].value_counts()

1     660
2     555
3     529
4     460
0     375
5     331
6     138
7      65
8      43
-1     15
       12
9       7
10      4
13      2
15      2
11      2
12      1
16      1
18      1
Name: Floor, dtype: int64

## 1.4. Extras
###### (Heating / Year Built / Orientation / AC / Terrace / Balcony / Parking / Temporary Stays / Furnished / Energy Certificate / State / Description / Idealista REF)

In [0]:
# HEATING

IR['Heating'] = IR['Text12']

def shift_values(row):
    if pd.notna(row['Heating']) and 'Riscal' not in row['Heating']:
        row['Heating'] = None 
    if pd.notna(row['Text13']) and 'Riscal' in row['Text13']:
        row['Heating'] = row['Text13']
    if pd.notna(row['Text14']) and 'Riscal' in row['Text14']:
        row['Heating'] = row['Text14']
    if pd.notna(row['Text15']) and 'Riscal' in row['Text15']:
        row['Heating'] = row['Text15']
    if pd.notna(row['Text16']) and 'Riscal' in row['Text16']:
        row['Heating'] = row['Text16']
    if pd.notna(row['Text17']) and 'Riscal' in row['Text17']:
        row['Heating'] = row['Text17']
    return row

IR = IR.apply(shift_values, axis=1)

def dummy(row):
    if row['Heating'] is not None and 'Riscaldamen' in row['Heating']:
        row['Heating'] = 1
    else:
        row['Heating'] = 0
    return row

IR = IR.apply(dummy, axis=1)

IR['Heating'].value_counts()

1    2986
0     452
Name: Heating, dtype: int64

In [0]:
# YEAR BUILT

IR['Year_Built'] = IR['Text12']

def shift_values(row):
    if pd.notna(row['Year_Built']) and 'Costruito nel ' not in row['Year_Built']:
        row['Year_Built'] = None 
    if pd.notna(row['Text13']) and 'Costruito nel ' in row['Text13']:
        row['Year_Built'] = row['Text13']
    if pd.notna(row['Text14']) and 'Costruito nel ' in row['Text14']:
        row['Year_Built'] = row['Text14']
    if pd.notna(row['Text15']) and 'Costruito nel ' in row['Text15']:
        row['Year_Built'] = row['Text15']
    if pd.notna(row['Text16']) and 'Costruito nel ' in row['Text16']:
        row['Year_Built'] = row['Text16']
    if pd.notna(row['Text17']) and 'Costruito nel ' in row['Text17']:
        row['Year_Built'] = row['Text17']
    return row

IR = IR.apply(shift_values, axis=1)

Year = {'Costruito nel ':''}

IR['Year_Built'].replace(Year, regex=True, inplace=True)

IR['Year_Built'].value_counts()

1900    478
1990    414
1930    301
1960    262
1970    181
       ... 
1928      1
1995      1
1957      1
1902      1
1929      1
Name: Year_Built, Length: 94, dtype: int64

In [0]:
# ORIENTATION

IR['Orientation'] = IR['Text12']

def shift_values(row):
    if pd.notna(row['Orientation']) and 'Orientamento' not in row['Orientation']:
        row['Orientation'] = None 
    if pd.notna(row['Text13']) and 'Orientamento' in row['Text13']:
        row['Orientation'] = row['Text13']
    if pd.notna(row['Text14']) and 'Orientamento' in row['Text14']:
        row['Orientation'] = row['Text14']
    if pd.notna(row['Text15']) and 'Orientamento' in row['Text15']:
        row['Orientation'] = row['Text15']
    if pd.notna(row['Text16']) and 'Orientamento' in row['Text16']:
        row['Orientation'] = row['Text16']
    if pd.notna(row['Text17']) and 'Orientamento' in row['Text17']:
        row['Orientation'] = row['Text17']
    return row

IR = IR.apply(shift_values, axis=1)

Orientamento = {'Orientamento ':''}

IR['Orientation'].replace(Orientamento, regex=True, inplace=True)

IR['Orientation'].value_counts()

sud                      539
est, ovest               224
sud, est                 167
est                      136
ovest                    111
nord, sud                109
sud, ovest                84
nord, sud, est, ovest     71
nord, est                 63
nord, ovest               54
sud, est, ovest           47
nord                      40
nord, est, ovest          19
nord, sud, est            18
nord, sud, ovest          15
Name: Orientation, dtype: int64

In [0]:
# TERRACE

IR['Terrace'] = IR['Text12']

def shift_values(row):
    if pd.notna(row['Terrace']) and ('Terrazzo' not in row['Terrace'] or 'Terrazza' not in row['Terrace']):
        row['Terrace'] = None 
    if pd.notna(row['Text13']) and ('Terrazzo' in row['Text13'] or 'Terrazza' in row['Text13']):
        row['Terrace'] = row['Text13']
    if pd.notna(row['Text14']) and ('Terrazzo' in row['Text14'] or 'Terrazza' in row['Text14']):
        row['Terrace'] = row['Text14']
    if pd.notna(row['Text15']) and ('Terrazzo' in row['Text15'] or 'Terrazza' in row['Text15']):
        row['Terrace'] = row['Text15']
    if pd.notna(row['Text16']) and ('Terrazzo' in row['Text16'] or 'Terrazza' in row['Text16']):
        row['Terrace'] = row['Text16']
    if pd.notna(row['Text17']) and ('Terrazzo' in row['Text17'] or 'Terrazza' in row['Text17']):
        row['Terrace'] = row['Text17']
    return row

IR = IR.apply(shift_values, axis=1)

def dummy(row):
    if pd.notna(row['Terrace']) and ('Terrazzo' in row['Terrace'] or 'Terrazza' in row['Terrace']):
        row['Terrace'] = 1
    else:
        row['Terrace'] = 0
    return row

IR = IR.apply(dummy, axis=1)

IR['Terrace'].value_counts()

0    3438
Name: Terrace, dtype: int64

In [0]:
#BALCONY

IR['Balcony'] = IR['Text12']

def shift_values(row):
    if pd.notna(row['Balcony']) and 'alcone' not in row['Balcony']:
        row['Balcony'] = None 
    if pd.notna(row['Text13']) and 'alcone' in row['Text13']:
        row['Balcony'] = row['Text13']
    if pd.notna(row['Text14']) and 'alcone' in row['Text14']:
        row['Balcony'] = row['Text14']
    if pd.notna(row['Text15']) and 'alcone' in row['Text15']:
        row['Balcony'] = row['Text15']
    if pd.notna(row['Text16']) and 'alcone' in row['Text16']:
        row['Balcony'] = row['Text16']
    if pd.notna(row['Text17']) and 'alcone' in row['Text17']:
        row['Balcony'] = row['Text17']
    return row

IR = IR.apply(shift_values, axis=1)

def dummy(row):
    if pd.notna(row['Balcony']) and 'alcone' in row['Balcony']:
        row['Balcony'] = 1
    else:
        row['Balcony'] = 0
    return row

IR = IR.apply(dummy, axis=1)

IR['Balcony'].value_counts()

0    3437
1       1
Name: Balcony, dtype: int64

In [0]:
#PARKING

IR['Parking'] = IR['Text17']

def shift_values(row):
    if pd.notna(row['Parking']) and 'arage' not in row['Parking']:
        row['Parking'] = None 
    if pd.notna(row['Text23']) and 'arage' in row['Text23']:
        row['Parking'] = row['Text23']
    return row

IR = IR.apply(shift_values, axis=1)

def dummy(row):
    if pd.notna(row['Parking']) and 'arage' in row['Parking']:
        row['Parking'] = 1
    else:
        row['Parking'] = 0
    return row

IR = IR.apply(dummy, axis=1)

IR['Parking'].value_counts()

0    3287
1     151
Name: Parking, dtype: int64

In [0]:
#TEMPORARY STAYS

IR['Temporart_Rentals'] = IR['Text24']

def shift_values(row):
    if pd.notna(row['Temporart_Rentals']) and 'Affitto stagionale' not in row['Temporart_Rentals']:
        row['Temporart_Rentals'] = None 
    return row

IR = IR.apply(shift_values, axis=1)

def dummy(row):
    if pd.notna(row['Temporart_Rentals']) and 'Affitto stagionale' in row['Temporart_Rentals']:
        row['Temporart_Rentals'] = 1
    else:
        row['Temporart_Rentals'] = 0
    return row

IR = IR.apply(dummy, axis=1)

IR['Temporart_Rentals'].value_counts()

0    3278
1     160
Name: Temporart_Rentals, dtype: int64

In [0]:
#FURNISHED

IR['Furnished'] = IR['Text12']

def shift_values(row):
    if pd.notna(row['Furnished']) and 'mobiliato' not in row['Furnished']:
        row['Furnished'] = None 
    if pd.notna(row['Text13']) and 'mobiliato' in row['Text13']:
        row['Furnished'] = row['Text13']
    if pd.notna(row['Text14']) and 'mobiliato' in row['Text14']:
        row['Furnished'] = row['Text14']
    if pd.notna(row['Text15']) and 'mobiliato' in row['Text15']:
        row['Furnished'] = row['Text15']
    if pd.notna(row['Text16']) and 'mobiliato' in row['Text16']:
        row['Furnished'] = row['Text16']
    if pd.notna(row['Text17']) and 'mobiliato' in row['Text17']:
        row['Furnished'] = row['Text17']
    return row

IR = IR.apply(shift_values, axis=1)

def dummy(row):
    if pd.notna(row['Furnished']) and 'mobiliato' in row['Furnished']:
        row['Furnished'] = 1
    else:
        row['Furnished'] = 0
    return row

IR = IR.apply(dummy, axis=1)

IR['Furnished'].value_counts()

1    2454
0     984
Name: Furnished, dtype: int64

In [0]:
#STATE

IR['State'] = IR['Text13']

def shift_values(row):
    if pd.notna(row['State']) and not any(x in row['State'] for x in ['Buono stato', 'Nuova costruzione', 'ristrutturare']):
        row['State'] = None
    if pd.notna(row['Text14']) and any(x in row['Text14'] for x in ['Buono stato', 'Nuova costruzione', 'ristrutturare']):
        row['State'] = row['Text14']
    if pd.notna(row['Text15']) and any(x in row['Text15'] for x in ['Buono stato', 'Nuova costruzione', 'ristrutturare']):
        row['State'] = row['Text15']
    if pd.notna(row['Text16']) and any(x in row['Text16'] for x in ['Buono stato', 'Nuova costruzione', 'ristrutturare']):
        row['State'] = row['Text16']
    if pd.notna(row['Text17']) and any(x in row['Text17'] for x in ['Buono stato', 'Nuova costruzione', 'ristrutturare']):
        row['State'] = row['Text17']
    return row

IR = IR.apply(shift_values, axis=1)

IR['State'].value_counts()

Buono stato                    1664
Da ristrutturare                  5
Nuova costruzione terminata       5
Nuova costruzione in corso        1
Name: State, dtype: int64

In [0]:
IR['Description'] = IR['Text23']

## 1.5. Final Dataset

In [0]:
IR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3438 entries, 0 to 3437
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Agency             3438 non-null   object 
 1   Text1              3438 non-null   object 
 2   District           3341 non-null   object 
 3   Neighborhood       3342 non-null   object 
 4   Street             3438 non-null   object 
 5   €_Sqm              3430 non-null   float64
 6   Monthly_Price      3431 non-null   float64
 7   Security_Deposit   0 non-null      float64
 8   Additional_Costs   2549 non-null   float64
 9   Elevator           3438 non-null   object 
 10  Floor              3203 non-null   object 
 11  Text12             3438 non-null   object 
 12  Text13             3438 non-null   object 
 13  Text14             3438 non-null   object 
 14  Text15             3438 non-null   object 
 15  Text16             3438 non-null   object 
 16  Text17             3438 

In [0]:
IR.drop(columns=['Text1', 'Text12', 'Text13', 'Text14', 'Text15', 'Text16', 'Text17', 'Text18', 'Text21', 'Text22', 'Text23', 'Text24', 'Text26', 'Text27', 'Text28', 'Text4', 'Text20', 'Text19'], inplace=True)

In [0]:
IR.tail(3)

,Agency,District,Neighborhood,Street,€_Sqm,Monthly_Price,Security_Deposit,Additional_Costs,Elevator,Floor,Bathrooms,Bedrooms,Constructed_Area,City,Heating,Year_Built,Orientation,Terrace,Balcony,Parking,Temporart_Rentals,Furnished,State,Description
3435,AGENZIA TROVA LA TUA CASA,ZONA PORTA VITTORIA,QUARTIERE CINQUE GIORNATE - CADORE,\nVIA ANDREA MAFFEI S.N.C\n,22.41,2600.0,NaN,400.0,1,2,2,4,116,MILAN,1,1970,None,0,0,0,0,1,Buono stato,R06- Via Maffei MM Porta Romana proponiamo in ...
3436,MASTERHOME SRL,ZONA NAVIGLI - BOCCONI,QUARTIERE SOLARI - SAVONA,"\nVIA MONTECATINI, 5\n",25.93,1400.0,NaN,200.0,1,4,1,2,54,MILAN,1,2021,est,0,0,1,0,1,Buono stato,
3437,VENDOCASA MILANO TIBALDI,ZONA NAVIGLI - BOCCONI,QUARTIERE NAVIGLI - PORTA GENOVA,"\nALZAIA NAVIGLIO PAVESE, 6\n",18.97,1100.0,NaN,100.0,1,1,1,2,58,MILAN,1,1930,None,0,0,0,0,1,None,


In [0]:
IR_Spark = spark.createDataFrame(IR)

In [0]:
IR_Spark.write.mode("overwrite").saveAsTable("silver.edgar_sarto_revenue.Idealista_CLEAN")

In [0]:
categoricas_1 = ['District']

categoricas_1 = pd.get_dummies(raw[categoricas_1], drop_first=False).astype(int)

categoricas_1